In [ ]:
import numpy as np

def generate_candidates(n_candidates, n_dim=4, lower=20.0, total_max=330.0, seed=None):

    rng = np.random.default_rng(seed)

    lower = float(lower)
    slack = total_max - n_dim * lower
    if slack <= 0:
        raise ValueError("Infeasible: total_max is too small for given lower bound.")

    alpha = np.ones(n_dim + 1)
    Y = rng.dirichlet(alpha, size=n_candidates)       # n_candidates, n_dim+1
    Z = Y[:, :n_dim] * slack                          # n_candidates, n_dim
    X = lower + Z                                     # candidates
    return X


def build_model_matrix_linear(X):
    #Linear model constraints

    n_samples, n_dim = X.shape
    intercept = np.ones((n_samples, 1))
    Phi = np.hstack([intercept, X])
    return Phi

def d_optimal_design(candidates, n_runs, seed=None):
    
    rng = np.random.default_rng(seed)

    N, d = candidates.shape
    Phi_all = build_model_matrix_linear(candidates)  # (N, p)
    p = Phi_all.shape[1]

    if n_runs < p:
        raise ValueError(
            f"Need at least {p} runs for full-rank linear model, but I got n_runs={n_runs}."
        )

    # Start with a random subset of size p
    initial_indices = rng.choice(N, size=p, replace=False)
    selected_indices = list(initial_indices)

    Phi_sel = Phi_all[selected_indices, :]

    # log(det(X'X))
    XtX = Phi_sel.T @ Phi_sel
    sign, logdet = np.linalg.slogdet(XtX)
    if sign <= 0:
        raise RuntimeError("Initial design matrix is not positive definite.")

    current_logdet = logdet


    used_mask = np.zeros(N, dtype=bool)
    used_mask[selected_indices] = True

    for _ in range(p, n_runs):
        best_logdet = -np.inf
        best_idx = None

        # adding each unused candidate and see which maximises log det(X'X)
        for idx in np.where(~used_mask)[0]:
            Phi_candidate = Phi_all[idx:idx+1, :]       # shape (1, p)
            Phi_trial = np.vstack([Phi_sel, Phi_candidate])
            XtX_trial = Phi_trial.T @ Phi_trial
            sign, logdet = np.linalg.slogdet(XtX_trial)
            if sign <= 0:
                continue  # skip non-PD
            if logdet > best_logdet:
                best_logdet = logdet
                best_idx = idx

        if best_idx is None:
            raise RuntimeError("Failed to find a candidate that improves det(X'X).")

        # Get best index
        selected_indices.append(best_idx)
        used_mask[best_idx] = True
        Phi_sel = np.vstack([Phi_sel, Phi_all[best_idx:best_idx+1, :]])
        current_logdet = best_logdet

    design_points = candidates[selected_indices, :]
    return design_points, selected_indices


def generate_constrained_doptimal(n_runs, n_candidates=8000, n_dim=4, lower=20.0, total_max=330.0, seed=42):

    candidates = generate_candidates(
        n_candidates=n_candidates,
        n_dim=n_dim,
        lower=lower,
        total_max=total_max,
        seed=seed,
    )
    design, indices = d_optimal_design(
        candidates=candidates,
        n_runs=n_runs,
        seed=seed,
    )
    return design




n_runs = 60
design = generate_constrained_doptimal(n_runs=n_runs, n_candidates=8000, n_dim=4, lower=20.0, total_max=330.0, seed=42)

print("First 5:\n", design[:5])
print("Min-Max per column:", design.min(axis=0), design.max(axis=0))

In [3]:
import pandas as pd
df = pd.DataFrame(np.round(design))
print(df)
df.to_csv("D_Optimal_raw.csv", header=False, index=False)

        0      1      2      3
0    64.0   26.0  169.0   25.0
1   154.0   42.0   69.0   47.0
2   160.0   63.0   50.0   49.0
3    23.0   98.0   92.0   46.0
4   152.0   82.0   33.0   45.0
5    25.0   29.0   38.0  238.0
6    26.0   29.0   23.0   21.0
7    25.0  237.0   23.0   21.0
8    22.0   44.0  236.0   25.0
9    20.0   21.0   26.0  235.0
10  237.0   28.0   28.0   22.0
11   29.0   28.0   24.0   21.0
12   29.0  236.0   23.0   20.0
13   30.0   33.0  237.0   27.0
14   43.0   26.0   20.0  237.0
15  235.0   21.0   30.0   41.0
16   29.0  236.0   34.0   30.0
17   21.0   29.0   31.0   28.0
18   26.0   32.0  232.0   34.0
19  234.0   22.0   26.0   27.0
20   31.0   28.0   32.0  232.0
21   26.0  235.0   23.0   28.0
22   26.0   33.0   28.0   24.0
23   31.0   20.0  232.0   28.0
24  234.0   25.0   35.0   27.0
25   26.0   24.0   40.0  230.0
26   30.0  231.0   32.0   30.0
27   29.0   36.0   26.0   23.0
28   23.0   24.0  231.0   46.0
29   24.0   24.0   50.0  229.0
30  232.0   25.0   33.0   27.0
31   26.

In [3]:
print(np.sum(np.sum(design, axis=1) < 330))

60
